In [1]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

raw_data = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/epi_r.csv')

In [36]:
#removes any column that isnt binary (0, 1)
features = list(raw_data.columns[6:])
raw_data.head()

,title,rating,calories,protein,fat,sodium,#cakeweek,#wasteless,22-minute meals,3-ingredient recipes,...,yellow squash,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack week,turkey
0,"Lentil, Apple, and Turkey Wrap",2.500,426.0,30.0,7.0,559.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Boudin Blanc Terrine with Red Onion Confit,4.375,403.0,18.0,23.0,1439.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Potato and Fennel Soup Hodge,3.750,165.0,6.0,7.0,165.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Mahi-Mahi in Tomato Olive Sauce,5.000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Spinach Noodle Casserole,3.125,547.0,20.0,32.0,452.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
print(raw_data.apply(pd.value_counts))

                                                    title  rating  calories  \
0.0                                                   NaN  1836.0      13.0   
1.0                                                   NaN     NaN       5.0   
1.25                                                  NaN   164.0       NaN   
1.875                                                 NaN   124.0       NaN   
2.0                                                   NaN     NaN      12.0   
...                                                   ...     ...       ...   
Zucchini-Pecan Cake with Cream Cheese Frosting        1.0     NaN       NaN   
Zucchini-Wrapped Halibut "Scallops"                   1.0     NaN       NaN   
Zucchini-Wrapped Red Snapper with Tomato, Cumin...    1.0     NaN       NaN   
Zuni Rolls with Raspberry Chipotle Sauce              1.0     NaN       NaN   
Zuppa di Cavolo Nero, Cannellini, e Salsicce: K...    1.0     NaN       NaN   

                                                   

In [110]:
remove_sub_10 = []
for col in features:
    if raw_data[col].sum() < 100:
        remove_sub_10.append(col)

#picks only features that were used over 1000 times      
feature_count = raw_data[features].sum().sort_values(ascending = False)
top_features = feature_count[feature_count >= 1000]

top_features
feature_keep_50 = ['bon appétit', 'peanut free', 'soy free', 'tree nut free', 'vegetarian', 'gourmet',
                   'kosher', 'pescatarian', 'quick & easy', 'wheat/gluten-free', 'bake', 'summer',
                   'dessert', 'dairy free', 'side', 'no sugar added', 'winter', 'fall', 'dinner',
                   'sugar conscious', 'healthy', 'kidney friendly', 'onion', 'tomato', 'vegetable',
                   'sauté', 'milk/cream', 'fruit', 'vegan', 'kid-friendly', 'egg', 'spring', 'herb',
                   'garlic', 'salad', 'dairy', 'thanksgiving', 'appetizer', 'lunch', 'cheese', 
                   'chicken', 'roast', 'no-cook', 'soup/stew', 'cocktail party', 'ginger', 'potato', 
                   'chill', 'lemon', 'grill/barbecue', 'drink', 'sauce', 'low cal', 'christmas', 'high fiber'
]
feature_keep_30 = feature_keep_50[:30]

In [107]:
# support vector regression for top 50 features
from sklearn.svm import SVR
svr = SVR()
X = raw_data[feature_keep_50].sample(frac=0.3, replace=True, random_state=1)
Y = raw_data.rating.sample(frac=0.3, replace=True, random_state=1)
svr.fit(X,Y)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [108]:
svr.score(X,Y)

0.23184162985945467

In [109]:
from sklearn.model_selection import cross_val_score
cross_val_score(svr, X, Y, cv=5)

array([0.09711461, 0.09085684, 0.09553449, 0.07860952, 0.09791605])

In [111]:
#support vector regression for top 30 features
svr = SVR()
X = raw_data[feature_keep_30].sample(frac=0.3, replace=True, random_state=1)
Y = raw_data.rating.sample(frac=0.3, replace=True, random_state=1)
svr.fit(X,Y)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [112]:
svr.score(X,Y)

0.10769037582377661

In [113]:
from sklearn.model_selection import cross_val_score
cross_val_score(svr, X, Y, cv=5)

array([0.00393343, 0.02068279, 0.02285987, 0.00924158, 0.01848035])

In [164]:
worth = []
for x in range (0, len(raw_data)):
    if raw_data.rating[x] < 4:
        worth.append(1)
    else:
        worth.append(0)
worth = pd.DataFrame(worth)
raw_data['worth'] = worth


,0
0,1
1,0
2,1
3,0
4,1
...,...
20047,1
20048,0
20049,0
20050,0


In [166]:
#SVC for top 50 features
from sklearn.svm import SVC

svm = SVC()
X = raw_data[feature_keep_50].sample(frac=0.3, replace=True, random_state=1)
Y = worth.sample(frac=0.3, replace=True, random_state=1)
svm.fit(X,Y.values.ravel(()))

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Non-string object detected for the array ordering. Please pass in 'C', 'F', 'A', or 'K' instead
  


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [167]:
svm.score(X,Y)

0.727061170212766

In [168]:
cross_val_score(svm, X, Y, cv=5)

/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

array([0.589701  , 0.61014131, 0.58935993, 0.5752286 , 0.61346633])

,0
235,1
12172,1
5192,0
17289,0
10955,0
...,...
15578,1
3314,1
17437,1
6696,0


In [170]:
#SVC for top 30 features
svm = SVC()
X = raw_data[feature_keep_30].sample(frac=0.3, replace=True, random_state=1)
Y = worth.sample(frac=0.3, replace=True, random_state=1)
svm.fit(X,Y.values.ravel(()))

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Non-string object detected for the array ordering. Please pass in 'C', 'F', 'A', or 'K' instead
  """


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [171]:
svm.score(X,Y)

0.668218085106383

In [172]:
cross_val_score(svm, X, Y, cv=5)

/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

array([0.58139535, 0.58104738, 0.57273483, 0.58686617, 0.57107232])